# E91 QKD

In [1]:
# Name: Brian Bahk
# Name: Minjae Cho
# Name: Elijah Olive

In [2]:
# Install qiskit
!pip install qiskit
!pip install pillow --upgrade

In [3]:
from google.colab import files
from PIL import Image
import io

uploaded = files.upload()

# Assuming you uploaded only one image, let's use that.
filename = next(iter(uploaded))
image = Image.open(io.BytesIO(uploaded[filename]))

Saving Train.png to Train.png


In [4]:
import numpy as np

# Convert image to grayscale
gray_image = image.convert("L")

# Convert grayscale image to binary (0 and 1)
threshold = 128  # Threshold to decide between 0 and 1
binary_image = np.array(gray_image) > threshold
binary_image = binary_image.astype(int)  # Convert boolean array to an int array (1s and 0s)

# Flatten the binary image to a 1D array
flat_binary_array = binary_image.flatten()
print(flat_binary_array)

[0 0 0 ... 0 0 0]


In [5]:
#@title #QC

from qiskit import QuantumCircuit
from qiskit.primitives.sampler import Sampler
import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class QC:

    def __init__(self):
        self.qc = QuantumCircuit(2, 2)

        # Create a Psi- state
        self.qc.h(1)
        self.qc.cx(1, 0)
        self.qc.x(1)
        self.qc.z(1)

        self.alice_alpha = 0
        self.alice_beta = 0
        self.bob_alpha = 0
        self.bob_beta = 0

    def prepareHV(self):
        # Add a measurement to both qubits
        self.qc.measure([0, 1], [0, 1])

        # Execute using the Sampler primitive
        sampler = Sampler()
        job = sampler.run(self.qc, shots=1)
        result = job.result()
        ab = int(str(result.quasi_dists[0])[1])
        if ab == 1:
            self.alice_alpha = 1
            self.alice_beta  = 0
            self.bob_alpha = 0
            self.bob_beta  = 1
        elif ab == 2:
            self.alice_alpha = 0
            self.alice_beta  = 1
            self.bob_alpha = 1
            self.bob_beta  = 0

    def prepareDA(self):
        # Put hadamard before measuring to measure in D/A basis
        self.qc.h(0)
        self.qc.h(1)

        # Add a measurement to both qubits
        self.qc.measure([0, 1], [0, 1])

        # Execute using the Sampler primitive
        sampler = Sampler()
        job = sampler.run(self.qc, shots=1)
        result = job.result()
        ab = int(str(result.quasi_dists[0])[1])
        # Psi- = 1/2sqrt2 * (|AD> - |DA>)
        if ab == 1: # 01; DA
            self.alice_alpha = 1/np.sqrt(2)
            self.alice_beta  = 1/np.sqrt(2)
            self.bob_alpha = 1/np.sqrt(2)
            self.bob_beta  = -1/np.sqrt(2)
        elif ab == 2: # 10; AD
            self.alice_alpha = 1/np.sqrt(2)
            self.alice_beta  = -1/np.sqrt(2)
            self.bob_alpha = 1/np.sqrt(2)
            self.bob_beta  = 1/np.sqrt(2)

    # Call after preparing (aka measuring one of the qubits)
    def getAliceKeyHV(self, probDarkCount):
        if self.alice_alpha == 1 and self.alice_beta == 0:
            return "H"
        elif self.alice_alpha == 0 and self.alice_beta == 1:
            return "V"
        else:
            return "N"

    def getAliceKeyDA(self, probDarkCount):
        if self.alice_alpha == 1/np.sqrt(2) and self.alice_beta == 1/np.sqrt(2):
            return "D"
        elif self.alice_alpha == 1/np.sqrt(2) and self.alice_beta == -1/np.sqrt(2):
            return "A"
        else:
            return "N"

    def measureHV(self, probDarkCount):
        if self.bob_alpha == 1 and self.bob_beta == 0:
            return "H"
        elif self.bob_alpha == 0 and self.bob_beta == 1:
            return "V"
        else:
            return "N"

    def measureDA(self, probDarkCount):
        if self.bob_alpha == 1/np.sqrt(2) and self.bob_beta == 1/np.sqrt(2):
            return "D"
        elif self.bob_alpha == 1/np.sqrt(2) and self.bob_beta == -1/np.sqrt(2):
            return "A"
        else:
            return "N"

    def eveSendH(self):
        self.bob_alpha = 1
        self.bob_beta  = 0

    def eveSendV(self):
        self.bob_alpha = 0
        self.bob_beta  = 1

    def eveSendD(self):
        self.bob_alpha = 1/np.sqrt(2)
        self.bob_beta  = 1/np.sqrt(2)

    def eveSendA(self):
        self.bob_alpha = 1/np.sqrt(2)
        self.bob_beta  = -1/np.sqrt(2)
    def bell_measurement(self, basis_a, basis_b):
        # Reset the circuit to start fresh for each measurement
        self.qc.reset([0, 1])

        # Prepare the Psi- state
        self.qc.h(1)
        self.qc.cx(1, 0)
        self.qc.x(1)
        self.qc.z(1)

        # Apply basis change depending on input
        if basis_a == 'D' or basis_a == 'A':
            self.qc.h(0)  # Hadamard on Alice's qubit for D/A basis
        if basis_b == 'D' or basis_b == 'A':
            self.qc.h(1)  # Hadamard on Bob's qubit for D/A basis

        # Measure
        self.qc.measure([0, 1], [0, 1])

        # Execute the circuit
        sampler = Sampler()
        job = sampler.run(self.qc, shots=1)
        result = job.result()
        ab = int(str(result.quasi_dists[0])[1])  # Extract the result
        return ab
    def calculate_correlations(self, shots=1000):
      settings = [('H', 'V'), ('H', 'D'), ('D', 'V'), ('D', 'D')]
      results = {setting: 0 for setting in settings}
      for _ in range(shots):
        for setting in settings:
            results[setting] += self.bell_measurement(*setting)
      return results

In [6]:
#n = 1000 # number of photons
n = len(flat_binary_array) *

SyntaxError: invalid syntax (<ipython-input-6-76e0e7d6a1ec>, line 2)

In [ ]:
# Alice chooses the encoding basis for each key bit.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisAlice = ""
for index in range(n):
  if random.randint(0,1) == 0:
    basisAlice += "+"
  else:
    basisAlice += "x"
print("basisAlice  = " + basisAlice)

In [ ]:
# Alice measures Bell State based on chosen basis
qcArray = [QC() for i in range(n)]
for i in range(n):
  if basisAlice[i] == "+":
    QC.prepareHV(qcArray[i])
  elif basisAlice[i] == "x":
    QC.prepareDA(qcArray[i])

In [ ]:
# Alice gets the raw key.
polarizationAlice = ""
for i in range(n):
  if basisAlice[i] == "+":
    polarizationAlice += QC.getAliceKeyHV(qcArray[i], .5)
  elif basisAlice[i] == "x":
    polarizationAlice += QC.getAliceKeyDA(qcArray[i], .5)
print("keyAlice  = " + polarizationAlice)

In [ ]:
# Alice infers the raw key.
# keyAlice should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement
keyAlice = ""
# TODO: Put your code here.
for i in range(n):
    if basisAlice[i] == '+':
        if polarizationAlice[i] == 'H':
            keyAlice += '0'
        elif polarizationAlice[i] == 'V':
            keyAlice += '1'
        else:
            keyAlice += '-'
    elif basisAlice[i] == 'x':
        if polarizationAlice[i] == 'D':
            keyAlice += '0'
        elif polarizationAlice[i] == 'A':
            keyAlice += '1'
        else:
            keyAlice += '-'
print("keyAlice      = " + keyAlice)

In [ ]:
#encrupt the data
#encruptedImage = flat_binary_array ^ keyAlice
encruptedImage = []
for i in range():
  if flat_binary_array[i] == keyAlice[i]:
        encruptedImage.append(0)  # XOR of same bits is 0
  else:
        encruptedImage.append(1)  # XOR of different bits is 1



In [ ]:
# Eve selects a subsample of photons from Alice to measure.
# interceptIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=intercepted
interceptIndex = ""
# TODO: Put your code here.
for i in range(n):
  if random.randint(0,1) == 0:
    interceptIndex += "0"
  else:
    interceptIndex += "1"
print(interceptIndex)

In [ ]:
# Eve chooses a basis to measure each intercepted photon.
# basisEve should be a string of n characters.
# Use the convention '+'=H/V, 'x'=D/A, ' '=not measured
basisEve = ""
# TODO: Put your code here.
for i in range(n):
  if interceptIndex[i] == '1':
    if random.randint(0,1) == 0:
      basisEve += "+"
    else:
      basisEve += "x"
  else:
    basisEve += ' '
print(basisEve)

In [ ]:
# Eve performs a measurement on each photon.
# outcomeEve should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeEve = ""
# TODO: Put your code here.
for i in range(n):
    if basisEve[i] == '+':
        outcomeEve += QC.measureHV(qcArray[i], .5)
    elif basisEve[i] == 'x':
        outcomeEve += QC.measureDA(qcArray[i], .5)
    else:
      outcomeEve += ' '
print(outcomeEve)

In [ ]:
# Eve resends photons to Bob.
# Be sure to handle the cases in which Eve gets an invalid measurement.
# TODO: Put your code here.
for i in range(n):
  if outcomeEve[i] == "H":
    QC.eveSendH(qcArray[i])
  elif outcomeEve[i] == "V":
    QC.eveSendV(qcArray[i])
  elif outcomeEve[i] == "D":
    QC.eveSendD(qcArray[i])
  elif outcomeEve[i] == "A":
    QC.eveSendA(qcArray[i])
  elif outcomeEve[i] == "N" or outcomeEve[i] == 'M':
    chooseRand = random.randint(0, 3)
    if chooseRand == 0:
      QC.eveSendH(qcArray[i])
    elif chooseRand == 1:
      QC.eveSendV(qcArray[i])
    elif chooseRand == 2:
      QC.eveSendD(qcArray[i])
    elif chooseRand == 3:
      QC.eveSendA(qcArray[i])
    #send random if N also choose based off of outcome if blank no change

In [ ]:
# Bob   --------------------------------------------

# Bob chooses a basis to measure each photon.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisBob = ""
for index in range(n):
  if random.randint(0,1) == 0:
    basisBob += "+"
  else:
    basisBob += "x"

print("basisBob    = " + basisBob)

In [ ]:
# Bob performs a measurement on each photon.
# Use the methods of the Photon class to measure each photon.
# outcomeBob should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeBob = ""
for i in range(n):
    if basisBob[i] == '+':
        outcomeBob += QC.measureHV(qcArray[i], .5)
    elif basisBob[i] == 'x':
        outcomeBob += QC.measureDA(qcArray[i], .5)
print("outcomeBob  = " + outcomeBob)

In [ ]:
# Bob infers the raw key.
# keyBob should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement

# Modified to accomodate Psi-
keyBob = ""
for i in range(n):
    if basisBob[i] == '+':
        if outcomeBob[i] == 'H':
            keyBob += '1'
        elif outcomeBob[i] == 'V':
            keyBob += '0'
        else:
            keyBob += '-'
    elif basisBob[i] == 'x':
        if outcomeBob[i] == 'D':
            keyBob += '1'
        elif outcomeBob[i] == 'A':
            keyBob += '0'
        else:
            keyBob += '-'
print("keyBob      = " + keyBob)

In [ ]:
# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# Bob also announces which of his measurements were invalid.
# -----------------------------------------------------------

In [ ]:
# Alice & Bob ----------------------------------------------------------

# Alice and Bob extract their sifted keys.
# siftedAlice and siftedBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.
for index in range(n):
    if keyAlice[index] == '-' or keyBob[index] == '-':
        siftedAlice += ' '
        siftedBob += ' '
    elif basisAlice[index] != basisBob[index]:
      siftedAlice += ' '
      siftedBob += ' '
    else:
        siftedAlice += keyAlice[index]
        siftedBob += keyBob[index]
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)

In [ ]:
# Alice and Bob use a portion of their sifted keys to estimate the quantum bit error rate (QBER).
# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampleIndex = ""
sampledBobQBER = 0
# TODO: Put your code here.
numError = 0
numTotal = 0

# out of non-removed sifted key, 50/50 chance to sample it
for i in range(n):
    if siftedAlice[i] == ' ' or random.randint(0, 1) == 0:
        sampleIndex += '0'
    else:
        sampleIndex += '1'
        numTotal += 1

#if sampling and it does not match add one to the number of errors
for i in range(n):
    if sampleIndex[i] == '1' and siftedAlice[i] != siftedBob[i]:
       numError += 1

sampledBobQBER = numError / numTotal
print(str(sampledBobQBER) + " QBER")
print("sampleIndex = "+sampleIndex)

In [ ]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""
# TODO: Put your code here.
for i in range (n):
  # if we sample it is removed
  if sampleIndex[i] == '0':
    secureAlice += ' '
    secureBob += ' '
  #if we don't sample we just pass it on
  else:
    secureAlice += siftedAlice[i]
    secureBob += siftedBob[i]
print("secureAlice = " +secureAlice)
print("secureBob =   "  + secureBob)

In [ ]:
# Alice and Bob make a hard determination whether the channel is secure.
# If it looks like there's something fishy, better hit the kill switch!
channelSecure = True # default value, to be changed to False if Eve suspected
# TODO: Put your code here.
if (sampledBobQBER > .0001):
  channelSecure = False

In [ ]:
# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured

# Also adjusted for Psi-
keyEve = ""
for index in range(len(outcomeEve)):
  if outcomeEve[index] == "H":
    keyEve += "1"
  elif outcomeEve[index] == "V":
    keyEve += "0"
  elif outcomeEve[index] == "D":
    keyEve += "1"
  elif outcomeEve[index] == "A":
    keyEve += "0"
  elif outcomeEve[index] == "N" or outcomeEve[index] == "M":
    keyEve += "-"
  else:
    keyEve += " "

print("keyEve     = " + keyEve)

In [ ]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publically revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the '0', '1', ' '=removed
stolenEve = ""
# TODO: Put your code here.
# i have keyalice and keybob
# i add
for i in range(n):
    if keyEve[i] == '-' or keyEve[i] == ' ' or basisAlice[i] != basisEve[i]:
        stolenEve += ' '
    else:
        stolenEve += keyAlice[i]
print("stolenEve = " + stolenEve)

In [ ]:
# ANALYSIS -------------------------------------------------------------

# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
       secureKeyLengthBob += 1
       if secureAlice[i] == secureBob[i]:
           numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1-actualBobQBER) * secureKeyLengthBob / n * 1e6;

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
       stolenKeyLengthEve += 1
       if secureAlice[i] == stolenEve[i]:
           numMatchEve += 1
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1-actualEveQBER) * stolenKeyLengthEve / n * 1e6;


# DISPLAY RESULTS ------------------------------------------------------

print("")
print("basisAlice  = " + basisAlice)
print("basisBob    = " + basisBob)
print("basisEve    = " + basisEve)
print("")
print("keyAlice    = " + keyAlice)
print("keyBob      = " + keyBob)
print("keyEve      = " + keyEve)
print("")
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)
print("")
print("secureAlice = " + secureAlice)
print("secureBob   = " + secureBob)
print("stolenEve   = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob/1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve/1000) + " kbps")

# Your goal is to maximize secureKeyRateBob and minimize stolenKeyRateEve.

In [ ]:
#decrupt image


unencruptedArray = encruptedImage
finalImage = []
for i in range(n):
        if keyBob[i] == encruptedImage[i]:
            finalImage.append(0)  # XOR of same bits is 0
        else:
            finalImage.append(1)  # XOR of different bits is 1

In [ ]:
# Reshape the 1D array back to the 2D array shape of the original binary image
originalshape = (160,215)


final_image_array = np.array(finalImage, dtype=np.uint8)
finalBad = np.array(unencruptedArray,dtype=np.uint8)
# Reshape the NumPy array to the original image dimensions
finalmage = final_image_array.reshape(originalshape)
finalBadimage = finalBad.reshape(originalshape)

# Convert the binary image back to a PIL image
# Multiplying by 255 because PIL expects pixel values from 0 to 255 for grayscale images
final_image = Image.fromarray((finalmage * 255).astype('uint8'))
final_wrong_image = Image.fromarray((finalBadimage * 255).astype('uint8'))

In [ ]:
import matplotlib.pyplot as plt

# Display the original and final binary images
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(final_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Binary Image')
plt.imshow(final_wrong_image, cmap='gray')
plt.axis('off')

plt.show()
